In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!unzip '/content/drive/MyDrive/Creating dataset.zip' -d '/content/data'

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/train_images/2d305cc3a.jpg  
  inflating: /content/data/train_images/2d31a59f2.jpg  
  inflating: /content/data/train_images/2d31c108a.jpg  
  inflating: /content/data/train_images/2d321711c.jpg  
  inflating: /content/data/train_images/2d338ba7c.jpg  
  inflating: /content/data/train_images/2d35b5518.jpg  
  inflating: /content/data/train_images/2d36dd423.jpg  
  inflating: /content/data/train_images/2d386cbb2.jpg  
  inflating: /content/data/train_images/2d3a2a31e.jpg  
  inflating: /content/data/train_images/2d3eee0e2.jpg  
  inflating: /content/data/train_images/2d402a054.jpg  
  inflating: /content/data/train_images/2d4100495.jpg  
  inflating: /content/data/train_images/2d412a8b7.jpg  
  inflating: /content/data/train_images/2d4240f1c.jpg  
  inflating: /content/data/train_images/2d42828e3.jpg  
  inflating: /content/data/train_images/2d44541ea.jpg  
  inflating: /content/data/train_images/2d458c653.jpg

In [1]:
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
from os.path import join
from keras.losses import binary_crossentropy
import keras.backend as K
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose, Dropout
from keras.optimizers import Adam
from skimage.io import imread
from skimage.transform import resize
from skimage import img_as_ubyte
import tensorflow as tf
from IPython.display import clear_output
from tensorflow.python.keras.metrics import MeanIoU

2023-06-13 20:57:45.629060: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 20:57:46.387793: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64
2023-06-13 20:57:46.387934: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64
2023-06-13 20:

In [2]:
DATA_PATH   = ''
TRAIN_PATH  = DATA_PATH+'train_images/'
TEST_PATH   = DATA_PATH+'train_images/' 
IMG_SIZE    = (768, 768)
INPUT_SHAPE = (768, 768, 3)
TARGET_SIZE = (384, 384)
TARGET_SIZE_RGB = (384, 384, 3)
BATCH_SIZE  = 4
EPOCHS      = 100

In [3]:
train_df = pd.read_csv('train_file.csv')

In [4]:
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

def masks_as_image(in_mask_list, all_masks=None):
    # Take the individual ship masks and create a single mask array for all ships
    if all_masks is None:
        all_masks = np.zeros((768, 768), dtype = np.int16)
    #if isinstance(in_mask_list, list):
    for mask in in_mask_list:
        if isinstance(mask, str):
            all_masks += rle_decode(mask)
    return np.expand_dims(all_masks, -1)

In [5]:
# Get Mask by Id and dataframe
def get_mask(img_id, df):
  img = masks_as_image(df.query('ImageId=="'+img_id+'"')['EncodedPixels'])
  img = resize(img, TARGET_SIZE, mode='constant', preserve_range=True)
  return img 

In [6]:
# Get Image by Id and dataframe
def get_image(image_name):
    img = imread(TRAIN_PATH + image_name)
    img = resize(img, (384, 384))
    return img_as_ubyte(img)

In [7]:
def make_image_gen(in_df, batch_size=BATCH_SIZE, VALIDATION=False):
    all_batches = list(in_df.groupby('ImageId'))
    masks = []
    images = []
    while True:
        np.random.shuffle(all_batches)
        for image, masks_df in all_batches:
            image, mask = [get_image(image)], [get_mask(image, masks_df)]
            images += image
            masks += mask
            if len(images)>=batch_size:
                yield np.stack(images, 0)/255.0, np.stack(masks, 0)
                masks, images=[], []

In [8]:
from tensorflow.python.ops.numpy_ops import np_config

np_config.enable_numpy_behavior()

# Dice coefficient
def dice_coef(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    y_true_f = y_true_f.astype(np.float32)
    y_pred_f = y_pred_f.astype(np.float32)

    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


In [9]:

from keras.models import Model
from keras.applications import ResNet50

from keras.layers import UpSampling2D, Concatenate, LeakyReLU, Dense

def decoder_block(x, y, filters):
    x = UpSampling2D()(x)
    x = Concatenate(axis = 3)([x,y])
    x = Conv2D(filters, 3, padding= 'same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Conv2D(filters, 3, padding= 'same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)

    return x

def resnet50_unet(input_shape, *, classes, dropout):
    """ Input """
    inputs = Input(input_shape)

    resnet50 = ResNet50(include_top=False, weights="imagenet", input_tensor=inputs)
    resnet50.trainable = False
    """ Encoder """
    s1 = resnet50.get_layer("input_1").output
    s2 = resnet50.get_layer("conv1_relu").output       
    s3 = resnet50.get_layer("conv2_block3_out").output 
    s4 = resnet50.get_layer("conv3_block4_out").output  

    x = resnet50.get_layer("conv4_block6_out").output  

    """ Decoder """
    x = decoder_block(x, s4, 512)                     
    x = decoder_block(x, s3, 256)                    
    x = decoder_block(x, s2, 128)                    
    x = decoder_block(x, s1, 64)                      

    x = Dropout(dropout)(x)
    outputs = Conv2D(classes, 1, activation="softmax")(x)

    model = Model(inputs, outputs, name="ResNet50_U-Net")
    return model

model = resnet50_unet((384, 384, 3), classes= 1, dropout= 0.2)
model.summary()

2023-06-13 20:57:47.847764: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64
2023-06-13 20:57:47.847797: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "ResNet50_U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 384, 384, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 390, 390, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 192, 192, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                     

In [10]:
import keras
keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [11]:
from keras.callbacks import ModelCheckpoint
weight_path="{}segmantation_weights.best.hdf5".format('model')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)

callbacks_list = [checkpoint]

In [12]:
train_df = train_df.dropna()
unique_masks = train_df.drop_duplicates('ImageId')

from sklearn.model_selection import train_test_split
train_df_l, validate_df_l = train_test_split(unique_masks)

train_df_l = train_df[train_df['ImageId'].isin(train_df_l['ImageId'])]
validate_df_l = train_df[train_df['ImageId'].isin(validate_df_l['ImageId'])]

In [13]:
BATCH_SIZE = 4
train_gen = make_image_gen(train_df_l)
BATCH_SIZE = 16
validation_gen = next(make_image_gen(validate_df_l))

In [14]:
model.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coef, MeanIoU(2)])

In [ ]:
step_count = 50
history = model.fit(train_gen,
                             steps_per_epoch=step_count,
                             epochs=100,
                             validation_data=validation_gen,
                             callbacks=callbacks_list)

Epoch 1/100
50/50 [==============================] - ETA: 0s - loss: 0.9895 - dice_coef: 0.0105 - mean_io_u: 0.4981 
Epoch 1: val_loss improved from inf to 0.96671, saving model to modelsegmantation_weights.best.hdf5
50/50 [==============================] - 595s 12s/step - loss: 0.9895 - dice_coef: 0.0105 - mean_io_u: 0.4981 - val_loss: 0.9667 - val_dice_coef: 0.0333 - val_mean_io_u: 0.4979
Epoch 2/100
50/50 [==============================] - ETA: 0s - loss: 0.9906 - dice_coef: 0.0094 - mean_io_u: 0.4980 
Epoch 2: val_loss did not improve from 0.96671
50/50 [==============================] - 584s 12s/step - loss: 0.9906 - dice_coef: 0.0094 - mean_io_u: 0.4980 - val_loss: 0.9667 - val_dice_coef: 0.0333 - val_mean_io_u: 0.4980
Epoch 3/100
50/50 [==============================] - ETA: 0s - loss: 0.9898 - dice_coef: 0.0102 - mean_io_u: 0.4980 
Epoch 3: val_loss did not improve from 0.96671
50/50 [==============================] - 583s 12s/step - loss: 0.9898 - dice_coef: 0.0102 - mean_io_u

In [ ]:
step_count = 50
history2 = model.fit(train_gen,
                             steps_per_epoch=step_count,
                             epochs=100,
                             validation_data=validation_gen,
                             callbacks=callbacks_list)

In [ ]:
step_count = 50
history = model.fit(train_gen,
                             steps_per_epoch=step_count,
                             epochs=100,
                             validation_data=validation_gen,
                             callbacks=callbacks_list)